In [58]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  # for plotting facilities

In [59]:
# todo aggregating regions together for easier model prediction

In [60]:
data = 'feature_metric.csv'

df = pd.read_csv(data)
df = df[['region', 'measure', 'epoch_', 'value',
       'Subject', 'Score_median_first20']]

In [ ]:
### agg cortex into 14 nodes

In [71]:
r = df.region.drop_duplicates()
r

0       7Networks_LH_Vis_1
1       7Networks_LH_Vis_2
2       7Networks_LH_Vis_3
3       7Networks_LH_Vis_4
4       7Networks_LH_Vis_5
               ...        
1035             Vermis IX
1036              Right IX
1037                Left X
1038              Vermis X
1039               Right X
Name: region, Length: 1040, dtype: object

In [88]:
cortex = r[:998].apply(lambda x: x[10:].split('_'))
cortex = cortex.tolist()
cortex = pd.DataFrame(cortex, columns=['hemisphere', 'network', 'position', 'num'])
cortex['num'] = cortex.apply(lambda r: r['position'] if not r['num'] else r['num'], axis=1)
cortex['position'] = cortex.apply(lambda r: None if not r['num'] != r['position'] else r['position'], axis=1)
cortex

,hemisphere,network,position,num
0,LH,Vis,None,1
1,LH,Vis,None,2
2,LH,Vis,None,3
3,LH,Vis,None,4
4,LH,Vis,None,5
...,...,...,...,...
993,RH,Default,pCunPCC,16
994,RH,Default,pCunPCC,17
995,RH,Default,pCunPCC,18
996,RH,Cont,pCun,2


In [89]:
cortex.describe()

,hemisphere,network,position,num
count,998,998,643,998
unique,2,7,23,103
top,LH,Default,Post,1
freq,500,212,102,49


In [99]:
subc = r[998:].apply(lambda x: x.split(' '))
subc = subc.to_list()
subc = pd.DataFrame(subc, columns=['hemisphere', 'network', 'position'])
subc['num'] = None
subc.describe()

,hemisphere,network,position,num
count,42,42,6,0
unique,3,16,2,0
top,Left,Crus,I,NaN
freq,17,6,3,NaN


In [106]:
cortex['r'] = 'cor_' + cortex['hemisphere'] + '_' + cortex['network']
subc['r'] = 'subc_' + subc['hemisphere']
# 14 regions from cortex, 3 regions from subc

In [113]:
r

0       7Networks_LH_Vis_1
1       7Networks_LH_Vis_2
2       7Networks_LH_Vis_3
3       7Networks_LH_Vis_4
4       7Networks_LH_Vis_5
               ...        
1035             Vermis IX
1036              Right IX
1037                Left X
1038              Vermis X
1039               Right X
Name: region, Length: 1040, dtype: object

In [121]:
regions_agg = pd.concat([cortex.r, subc.r], axis=0).reset_index(drop=True)
regions_agg = pd.concat([regions_agg.rename('region_agg'), r], axis=1)
regions_agg

,region_agg,region
0,cor_LH_Vis,7Networks_LH_Vis_1
1,cor_LH_Vis,7Networks_LH_Vis_2
2,cor_LH_Vis,7Networks_LH_Vis_3
3,cor_LH_Vis,7Networks_LH_Vis_4
4,cor_LH_Vis,7Networks_LH_Vis_5
...,...,...
1035,subc_Vermis,Vermis IX
1036,subc_Right,Right IX
1037,subc_Left,Left X
1038,subc_Vermis,Vermis X


In [130]:
dfecc = df[df['measure'] == 'eccentricity']
dfecc = dfecc.merge(regions_agg, on='region')

In [133]:
# dfecc.groupby(['Subject', 'measure', 'epoch_', 'region_agg'])['value'].mean()
# by default, pivot_table takes mean

Subject  measure       epoch_  region_agg        
AB1      eccentricity  b2e     cor_LH_Cont           0.096832
                               cor_LH_Default        0.285795
                               cor_LH_DorsAttn       0.336393
                               cor_LH_Limbic         0.191118
                               cor_LH_SalVentAttn    0.141011
                                                       ...   
ZN1      eccentricity  late    cor_RH_SomMot         2.245211
                               cor_RH_Vis            2.400097
                               subc_Left             0.873998
                               subc_Right            0.920408
                               subc_Vermis           0.869525
Name: value, Length: 3060, dtype: float64

In [134]:
dfecc = dfecc.pivot_table(columns=['epoch_', 'region_agg'],
               values=['value'],
               index=['Subject', 'Score_median_first20'])

dfecc

value                                 \
epoch_                               b2e                                  
region_agg                   cor_LH_Cont cor_LH_Default cor_LH_DorsAttn   
Subject Score_median_first20                                              
AB1     14.0                    0.096832       0.285795        0.336393   
AG1     47.0                    0.023247       0.134270        0.016678   
AH1     41.5                   -0.323308      -0.037860       -0.711544   
AM1     27.0                    0.175577       0.412751        0.288554   
AP1     17.0                    0.467508       0.169592       -0.193752   
AV1     48.5                   -0.038907      -0.278500        0.676340   
BN1     75.0                    0.056349      -0.016979       -0.353304   
CD1     15.0                    0.051840      -0.055279       -0.196941   
CG1     32.0                    0.108409       0.024720        0.175183   
CH1     39.0                    0.177342      -0.087421       -0.016997   
CK1     34.0                    0.086924       0.137110        0.299040   
CM1     29.5                    0.210084      -0.287603        0.282841   
CM2     73.0                    0.268694       0.294601       -0.253968   
EH1     21.0                    0.387339       0.257275        0.291328   
EK1     12.0                    0.013718      -0.145739        0.582988   
HK1     17.0                    0.321216       0.582425        0.415549   
HS1     67.0                    0.170226      -0.168491        0.353160   
JC1     12.5                    0.441986       0.234091        0.437221   
JH1     8.5                    -0.026666       0.106128        0.104092   
JM1     34.0                   -0.281974      -0.076701        0.006779   
JR1     34.0                   -0.072512       0.251876       -0.125334   
JS1     57.0                    0.057821       0.119935        0.076817   
KK1     11.5                   -0.025902       0.484708        0.069643   
KP1     38.0                    0.131821       0.172627        0.242722   
KR1     25.0                    0.492554       0.689035        0.391546   
LB1     12.0                   -0.088604       0.422639        0.162973   
MB1     39.0                   -0.056847       0.236683       -0.310333   
MG1     57.0                   -0.252071      -0.571489       -0.340796   
MP1     45.0                    0.038101      -0.436873       -0.311200   
NW1     26.0                   -0.284932      -0.155696        0.231738   
SK1     39.0                    0.353440       0.344001       -0.212267   
SM2     44.0                    0.156058       0.257919        0.384763   
SR1     21.5                   -0.047300      -0.070251       -0.042661   
UP1     46.0                    0.141589       0.151221        0.142272   
WK1     62.0                    0.068048       0.233606        0.053965   
ZN1     55.0                    0.146824      -0.541973       -0.176711   

                                                                             \
epoch_                                                                        
region_agg                   cor_LH_Limbic cor_LH_SalVentAttn cor_LH_SomMot   
Subject Score_median_first20                                                  
AB1     14.0                      0.191118           0.141011      0.004024   
AG1     47.0                     -0.172098           0.584519      0.466026   
AH1     41.5                     -0.305593          -0.227577      0.013477   
AM1     27.0                      0.048740           0.684970      0.351564   
AP1     17.0                     -0.240978           0.309113     -0.226975   
AV1     48.5                     -0.182464           0.137662      0.267408   
BN1     75.0                     -0.461987          -0.095968     -0.302246   
CD1     15.0                     -0.095444          -0.239501     -0.159295   
CG1     32.0                     -0.050737          -0.377077     -0.248930   
CH1    

In [128]:
dfecc[('value',  'b2e', 'cor_LH_Cont')]

Subject  Score_median_first20
AB1      14.0                    0.096832
AG1      47.0                    0.023247
AH1      41.5                   -0.323308
AM1      27.0                    0.175577
AP1      17.0                    0.467508
AV1      48.5                   -0.038907
BN1      75.0                    0.056349
CD1      15.0                    0.051840
CG1      32.0                    0.108409
CH1      39.0                    0.177342
CK1      34.0                    0.086924
CM1      29.5                    0.210084
CM2      73.0                    0.268694
EH1      21.0                    0.387339
EK1      12.0                    0.013718
HK1      17.0                    0.321216
HS1      67.0                    0.170226
JC1      12.5                    0.441986
JH1      8.5                    -0.026666
JM1      34.0                   -0.281974
JR1      34.0                   -0.072512
JS1      57.0                    0.057821
KK1      11.5                   -0.025902
KP1 

In [135]:
dfecc

value                                 \
epoch_                               b2e                                  
region_agg                   cor_LH_Cont cor_LH_Default cor_LH_DorsAttn   
Subject Score_median_first20                                              
AB1     14.0                    0.096832       0.285795        0.336393   
AG1     47.0                    0.023247       0.134270        0.016678   
AH1     41.5                   -0.323308      -0.037860       -0.711544   
AM1     27.0                    0.175577       0.412751        0.288554   
AP1     17.0                    0.467508       0.169592       -0.193752   
AV1     48.5                   -0.038907      -0.278500        0.676340   
BN1     75.0                    0.056349      -0.016979       -0.353304   
CD1     15.0                    0.051840      -0.055279       -0.196941   
CG1     32.0                    0.108409       0.024720        0.175183   
CH1     39.0                    0.177342      -0.087421       -0.016997   
CK1     34.0                    0.086924       0.137110        0.299040   
CM1     29.5                    0.210084      -0.287603        0.282841   
CM2     73.0                    0.268694       0.294601       -0.253968   
EH1     21.0                    0.387339       0.257275        0.291328   
EK1     12.0                    0.013718      -0.145739        0.582988   
HK1     17.0                    0.321216       0.582425        0.415549   
HS1     67.0                    0.170226      -0.168491        0.353160   
JC1     12.5                    0.441986       0.234091        0.437221   
JH1     8.5                    -0.026666       0.106128        0.104092   
JM1     34.0                   -0.281974      -0.076701        0.006779   
JR1     34.0                   -0.072512       0.251876       -0.125334   
JS1     57.0                    0.057821       0.119935        0.076817   
KK1     11.5                   -0.025902       0.484708        0.069643   
KP1     38.0                    0.131821       0.172627        0.242722   
KR1     25.0                    0.492554       0.689035        0.391546   
LB1     12.0                   -0.088604       0.422639        0.162973   
MB1     39.0                   -0.056847       0.236683       -0.310333   
MG1     57.0                   -0.252071      -0.571489       -0.340796   
MP1     45.0                    0.038101      -0.436873       -0.311200   
NW1     26.0                   -0.284932      -0.155696        0.231738   
SK1     39.0                    0.353440       0.344001       -0.212267   
SM2     44.0                    0.156058       0.257919        0.384763   
SR1     21.5                   -0.047300      -0.070251       -0.042661   
UP1     46.0                    0.141589       0.151221        0.142272   
WK1     62.0                    0.068048       0.233606        0.053965   
ZN1     55.0                    0.146824      -0.541973       -0.176711   

                                                                             \
epoch_                                                                        
region_agg                   cor_LH_Limbic cor_LH_SalVentAttn cor_LH_SomMot   
Subject Score_median_first20                                                  
AB1     14.0                      0.191118           0.141011      0.004024   
AG1     47.0                     -0.172098           0.584519      0.466026   
AH1     41.5                     -0.305593          -0.227577      0.013477   
AM1     27.0                      0.048740           0.684970      0.351564   
AP1     17.0                     -0.240978           0.309113     -0.226975   
AV1     48.5                     -0.182464           0.137662      0.267408   
BN1     75.0                     -0.461987          -0.095968     -0.302246   
CD1     15.0                     -0.095444          -0.239501     -0.159295   
CG1     32.0                     -0.050737          -0.377077     -0.248930   
CH1    

In [43]:
X = dfecc.reset_index().drop(['Subject', 'Score_median_first20'], axis=1)

y = dfecc.reset_index()['Score_median_first20']

/var/folders/2q/y6tqndfj1yqfc7hcdp99tz_00000gn/T/ipykernel_12408/1566465353.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = dfecc.reset_index().drop(['Subject', 'Score_median_first20'], axis=1)


In [44]:
X

value                           \
epoch_                      b2e                            
region 7Networks_LH_Cont_Cing_1 7Networks_LH_Cont_Cing_2   
0                      0.703477                -0.927863   
1                      0.357591                -0.575116   
2                     -0.014645                -0.007817   
3                      0.363827                -0.455096   
4                      1.243632                 0.095185   
5                     -1.008693                 1.160782   
6                     -0.177668                -0.631989   
7                     -0.338410                -0.190189   
8                      0.026779                 0.418469   
9                      0.711932                 1.881008   
10                     0.206989                 0.368045   
11                     0.790264                -0.490761   
12                     0.014010                 0.586256   
13                     1.182094                 0.056378   
14                    -0.101774                -0.528102   
15                    -0.265762                 0.420327   
16                     0.059892                 0.633167   
17                    -0.626487                -0.158431   
18                    -0.866298                -0.098429   
19                    -0.218577                -0.386367   
20                     0.072564                 0.199435   
21                     0.187944                 0.021905   
22                     0.802707                 0.666476   
23                     1.286294                 0.139949   
24                    -0.264532                -0.343811   
25                    -0.117901                 0.176652   
26                     1.606390                -0.250953   
27                     0.402983                -0.098029   
28                     0.129998                -1.530500   
29                    -0.627209                -0.328759   
30                     0.701957                 0.473165   
31                     0.692927                -0.046108   
32                    -0.914123                -0.900474   
33                    -0.126119                 1.328713   
34                    -0.568768                -0.244131   
35                     0.675976                 0.855875   

                                                          \
epoch_                                                     
region 7Networks_LH_Cont_Cing_3 7Networks_LH_Cont_Cing_4   
0                      0.166523                -0.016294   
1                     -0.198989                -0.607698   
2                     -0.355955                -1.415309   
3                     -0.745145                 0.080842   
4                      1.172546                -1.218897   
5                     -0.221900                 0.478659   
6                      0.354508                -0.721208   
7                      0.074796                -0.044014   
8                      0.012713                 0.640000   
9                     -0.205136                 0.068042   
10                     0.451437                -0.305220   
11                    -0.296272                -1.581116   
12                    -0.973766                 0.215011   
13                    -0.676597                 0.491842   
14                    -0.076403                 0.872357   
15                     0.454712                 0.302573   
16                     0.761280                 0.760854   
17                    -0.676748                -0.803616   
18                    -0.573917                 0.431163   
19                    -0.161368                -0.327313   
20                     0.147559                -0.179394   
21                     0.226958                 0.127067   
22                    -0.064805                 0.980947   
23                    -0.433095                -0.043339   
24                    -0.577846                -0.340454   
25   

In [42]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()

# for f in X.columns:
    # X[f] = lbl.fit_transform(X[f].astype(str))

# y = lbl.fit_transform(y.astype(str))

In [45]:
# import XGBoost
import xgboost as xgb


# define data_dmatrix
data_dmatrix = xgb.DMatrix(data=X,label=y,
                           # enable_categorical=True
                           )

## **4.5 Split data into separate training and test set** <a class="anchor" id="4.5"></a>

[Table of Contents](#0.1)

In [46]:
# split X and y into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [47]:
y

0     14.0
1     47.0
2     41.5
3     27.0
4     17.0
5     48.5
6     75.0
7     15.0
8     32.0
9     39.0
10    34.0
11    29.5
12    73.0
13    21.0
14    12.0
15    17.0
16    67.0
17    12.5
18     8.5
19    34.0
20    34.0
21    57.0
22    11.5
23    38.0
24    25.0
25    12.0
26    39.0
27    57.0
28    45.0
29    26.0
30    39.0
31    44.0
32    21.5
33    46.0
34    62.0
35    55.0
Name: Score_median_first20, dtype: float64

In [48]:
y_test

31    44.0
20    34.0
16    67.0
30    39.0
22    11.5
15    17.0
10    34.0
2     41.5
11    29.5
29    26.0
27    57.0
Name: Score_median_first20, dtype: float64

In [ ]:
X_test

In [49]:
# import XGBClassifier
from xgboost import XGBClassifier


# declare parameters
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }         
           
          
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)


# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

XGBClassifier(alpha=10, learning_rate=1.0, max_depth=4,
              objective='multi:softprob')

In [50]:
# we can view the parameters of the xgb trained model as follows -

print(xgb_clf)

XGBClassifier(alpha=10, learning_rate=1.0, max_depth=4,
              objective='multi:softprob')


In [51]:
# make predictions on test data

y_pred = xgb_clf.predict(X_test)

In [52]:
y_test

31    44.0
20    34.0
16    67.0
30    39.0
22    11.5
15    17.0
10    34.0
2     41.5
11    29.5
29    26.0
27    57.0
Name: Score_median_first20, dtype: float64

In [53]:
y_pred

array([15., 15., 15., 39., 15., 12., 15., 15., 39., 39., 15.])

In [54]:
# compute and print accuracy score

from sklearn.metrics import mean_squared_error

print('XGBoost model mean sq error: {0:0.4f}'. format(mean_squared_error(y_test, y_pred)))

XGBoost model mean sq error: 639.0682


We can see that XGBoost obtain very high accuracy score of 91.67%.

# **5. k-fold Cross Validation using XGBoost** <a class="anchor" id="5"></a>


[Table of Contents](#0.1)


- To build more robust models with XGBoost, we should always perform k-fold cross validation. 

- In this way, we ensure that the original training dataset is used for both training and validation. 

- Also, each entry is used for validation just once. 

- XGBoost supports k-fold cross validation using the cv() method. 

- In this method, we will specify several parameters which are as follows:-

  - **nfolds** - This parameter specifies the number of cross-validation sets we want to build.

  - **num_boost_round** - It denotes the number of trees we build.

  - **metrics** - It is the performance evaluation metrics to be considered during CV.

  - **as_pandas** - It is used to return the results in a pandas DataFrame.

  - **early_stopping_rounds** - This parameter stops training of the model early if the hold-out metric does not improve for a given number of rounds.

  - **seed** - This parameter is used for reproducibility of results.

We can use these parameters to build a k-fold cross-validation model by calling XGBoost's CV() method.

In [27]:
from xgboost import cv

params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10, metrics="auc", as_pandas=True, seed=123)

XGBoostError: [14:03:41] src/objective/regression_obj.cu:101: label must be in [0,1] for logistic regression
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000016da1cc08 dmlc::LogMessageFatal::~LogMessageFatal() + 64
  [bt] (1) 2   libxgboost.dylib                    0x000000016da8f770 xgboost::obj::RegLossObj<xgboost::obj::LogisticClassification>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 920
  [bt] (2) 3   libxgboost.dylib                    0x000000016da19c1c xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 1004
  [bt] (3) 4   libxgboost.dylib                    0x000000016da32e18 XGBoosterUpdateOneIter + 212
  [bt] (4) 5   libffi.dylib                        0x00000001a7b7c050 ffi_call_SYSV + 80
  [bt] (5) 6   libffi.dylib                        0x00000001a7b84ae8 ffi_call_int + 1208
  [bt] (6) 7   _ctypes.cpython-38-darwin.so    

- **xgb_cv** contains train and test auc metrics for each boosting round. 

- Let's preview **xgb_cv**.

In [26]:
xgb_cv.head()

NameError: name 'xgb_cv' is not defined

# **6. Feature importance with XGBoost** <a class="anchor" id="6"></a>

[Table of Contents](#0.1)


- XGBoost provides a way to examine the importance of each feature in the original dataset within the model. 

- It involves counting the number of times each feature is split on across all boosting trees in the model. 

- Then we visualize the result as a bar graph, with the features ordered according to how many times they appear.

- XGBoost has a **plot_importance()** function that helps us to achieve this task. 

- Then we can visualize the features that has been given the highest important score among all the features. 

- Thus XGBoost provides us a way to do feature selection.

- We will proceed as follows:-

In [ ]:
xgb.plot_importance(xgb_clf)
plt.figure(figsize = (16, 12))
plt.show()

- We can see that the feature `Delicassesn` has been given the highest importance score among all the features. 

- Based upon this importance score, we can select the features with highest importance score and discard the redundant ones.

- Thus XGBoost also gives us a way to do feature selection.

# **7. Results and Conclusion** <a class="anchor" id="7"></a>

[Table of Contents](#0.1)


- In this kernel, we implement XGBoost with Python and Scikit-Learn to classify the customers from two different channels as Horeca (Hotel/Retail/Café) customers or Retail channel (nominal) customers.

- The y labels contain values as 1 and 2. We have converted them into 0 and 1 for further analysis.

- We have trained the XGBoost classifier and found the accuracy score to be 91.67%.

- We have performed k-fold cross-validation with XGBoost.

- We have find the most important feature in XGBoost. We did it using the plot_importance() function in XGBoost that helps us to achieve this task.



# **8. References** <a class="anchor" id="8"></a>

[Table of Contents](#0.1)


The ideas and concepts in this kernel are taken from the following websites -

-	https://www.datacamp.com/community/tutorials/xgboost-in-python

-	https://blog.cambridgespark.com/getting-started-with-xgboost-3ba1488bb7d4

-	https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

-	https://heartbeat.fritz.ai/boosting-your-machine-learning-models-using-xgboost-d2cabb3e948f

-	https://towardsdatascience.com/https-medium-com-vishalmorde-xgboost-algorithm-long-she-may-rein-edd9f99be63d

-   https://medium.com/@gabrieltseng/gradient-boosting-and-xgboost-c306c1bcfaf5

-   https://medium.com/mlreview/gradient-boosting-from-scratch-1e317ae4587d